In [1]:
import pandas as pd
import os
import json

In [2]:
data_path = r'E:\data\data_set_phase1'
test_queries_data_path = os.path.join(data_path,'test_queries.csv')
test_plans_data_path = os.path.join(data_path,'test_plans.csv')
profiles_data_path = os.path.join(data_path,'profiles.csv')

train_queries_data_path = os.path.join(data_path,'train_queries.csv')
train_plans_data_path = os.path.join(data_path,'train_plans.csv')
train_clicks_data_path = os.path.join(data_path,'train_clicks.csv')

In [3]:
df_test_queries = pd.read_csv(test_queries_data_path)
df_test_plans = pd.read_csv(test_plans_data_path)
df_profiles = pd.read_csv(profiles_data_path)

df_train_queries = pd.read_csv(train_queries_data_path)
df_train_plans = pd.read_csv(train_plans_data_path)
df_train_clicks = pd.read_csv(train_clicks_data_path)

In [4]:
df_train = pd.merge(df_train_queries,df_train_clicks,how='left',on=['sid'])
df_train = pd.merge(df_train, df_train_plans, how='left', on=['sid'])
df_train = pd.merge(df_train, df_profiles, how='left', on=['pid'])
df_train['click_mode'] = df_train.click_mode.fillna(0)
df_train = df_train.fillna(method='ffill')
df_train = df_train.fillna(method='bfill')
df_train = df_train.drop(['req_time','o','d','click_time','plan_time','pid'],axis=1) 
# time 仅考虑一天的时间，转成秒数 ，od经纬度转成两列，
# plan中price的填补，可以将一种mode里所有的dis和price做个y=wx+b的拟合？
df_train.count()

sid           500000
click_mode    500000
plans         500000
p0            500000
p1            500000
p2            500000
p3            500000
p4            500000
p5            500000
p6            500000
p7            500000
p8            500000
p9            500000
p10           500000
p11           500000
p12           500000
p13           500000
p14           500000
p15           500000
p16           500000
p17           500000
p18           500000
p19           500000
p20           500000
p21           500000
p22           500000
p23           500000
p24           500000
p25           500000
p26           500000
               ...  
p36           500000
p37           500000
p38           500000
p39           500000
p40           500000
p41           500000
p42           500000
p43           500000
p44           500000
p45           500000
p46           500000
p47           500000
p48           500000
p49           500000
p50           500000
p51           500000
p52          

In [6]:
plan_ls = list(df_train.plans)
plan_ls = [json.loads(p) for p in plan_ls]
# 填补price缺失值，对没有缺失的数据用 gbdt拟合 ，预测缺失的price
x = []
y = []
from sklearn.ensemble import GradientBoostingRegressor
gbdt = GradientBoostingRegressor()

for p in plan_ls:
    for pdc in p:
        if pdc['price']!="":
            dis = int(pdc['distance'])
            eta = int(pdc['eta'])
            mode = int(pdc['transport_mode'])
            price = int(pdc['price'])
            x.append([dis,eta,mode])
            y.append(price)
print('sta')
gbdt.fit(x,y)
print('fi')
for p in plan_ls:
    for pdc in p:
        if pdc['price']=="":
            dis = int(pdc['distance'])
            eta = int(pdc['eta'])
            mode = int(pdc['transport_mode'])
            pdc['price'] = gbdt.predict([[dis,eta,mode]])[0]
            
            

sta
fi


In [7]:
inf = 10000000
mode_dic = {}
for i in range(0,33):
    mode_dic[i]=[]
# for i in range(0,33):
#     mode_dic[i].append(inf)
# mode_dic
for p in plan_ls:
    for i in range(0,33):
        mode_dic[i].append(inf)
    for pdc in p:
            dis = int(pdc['distance'])
            eta = int(pdc['eta'])
            mode = int(pdc['transport_mode'])
            price = int(pdc['price'])
            mode_dic[(mode-1)*3+0][-1] = dis
            mode_dic[(mode-1)*3+1][-1] = eta
            mode_dic[(mode-1)*3+2][-1] = price


In [8]:
for i in range(0,33):
    df_train['plan'+str(i)] = mode_dic[i]
    mode_dic[i] = []

In [9]:
df_train['sid'] = df_train.sid.apply(int)

In [10]:
df_train['click_mode'] = df_train.click_mode.apply(int)

In [11]:
cols = list(df_train.columns)
for c in cols[3:69]:
    df_train[c] = df_train[c].apply(int)

In [12]:
def plan2mode(s):
    plan_lis = json.loads(s)
    pl = []
    for p in plan_lis:
        pl.append(int(p['transport_mode']))
    return pl
df_train['plans'] = df_train.plans.apply(plan2mode)
df_train.head()

,sid,click_mode,plans,p0,p1,p2,p3,p4,p5,p6,...,plan23,plan24,plan25,plan26,plan27,plan28,plan29,plan30,plan31,plan32
0,3000821,9,"[9, 3, 4, 6, 2, 5]",0,0,0,0,1,0,0,...,10000000,5219,1367,300,10000000,10000000,10000000,10000000,10000000,10000000
1,3085857,1,"[7, 3, 4, 9, 1]",0,0,0,0,1,0,0,...,10000000,13488,2315,400,10000000,10000000,10000000,10000000,10000000,10000000
2,2944522,9,"[9, 3, 4, 10, 2, 6, 7]",0,0,0,0,1,0,0,...,10000000,12294,2472,400,12996,2341,1700,10000000,10000000,10000000
3,559931,1,"[10, 3, 4, 7, 1]",1,0,0,0,0,0,0,...,10000000,10000000,10000000,10000000,14853,2914,1700,10000000,10000000,10000000
4,2819352,7,"[7, 3, 4, 7]",0,0,0,1,0,0,0,...,10000000,10000000,10000000,10000000,10000000,10000000,10000000,10000000,10000000,10000000


In [13]:
df_train.drop(['sid','plans','click_mode'],axis=1).head()

,p0,p1,p2,p3,p4,p5,p6,p7,p8,p9,...,plan23,plan24,plan25,plan26,plan27,plan28,plan29,plan30,plan31,plan32
0,0,0,0,0,1,0,0,0,1,1,...,10000000,5219,1367,300,10000000,10000000,10000000,10000000,10000000,10000000
1,0,0,0,0,1,0,0,0,1,1,...,10000000,13488,2315,400,10000000,10000000,10000000,10000000,10000000,10000000
2,0,0,0,0,1,0,0,0,1,1,...,10000000,12294,2472,400,12996,2341,1700,10000000,10000000,10000000
3,1,0,0,0,0,0,0,0,1,0,...,10000000,10000000,10000000,10000000,14853,2914,1700,10000000,10000000,10000000
4,0,0,0,1,0,0,0,1,0,1,...,10000000,10000000,10000000,10000000,10000000,10000000,10000000,10000000,10000000,10000000


In [17]:
click_mode = list(df_train.click_mode)

In [75]:
import xgboost as xgb
param = {}
param['num_class'] = 12
param['objective'] = 'multi:softprob'
# param = {}
# param['eta'] = 0.1
# param['max_depth'] = 6
# param['silent'] = 1
# param['nthread'] = 4
# param['num_class'] = 3
# num_round = 50
# param['objective'] = 'multi:softprob'
num_round = 20
bst = xgb.train(param,xgb.DMatrix(df_train.drop(['sid','plans','click_mode'],axis=1).values,label=click_mode),num_round)

In [76]:
pred = bst.predict(xgb.DMatrix(df_train.drop(['sid','plans','click_mode'],axis=1).values))


In [77]:
import numpy as np
pred = pred.tolist()

In [78]:
pred_label = []
# sorted(list(zip(range(0,12),pred[0])),key = lambda x:x[1],reverse=True)
for i in range(len(pred)):
    sort_pred = sorted(list(zip(range(0,12),pred[i])),key = lambda x:x[1],reverse=True)
    for prob in sort_pred:
        if prob[0] in df_train['plans'][i] or prob[0] == 0:
            pred_label.append(prob[0])
            break

In [79]:
len(pred_label)

500000

In [80]:
pro = [pred_label[i]==click_mode[i] for i in range(len(pred_label))]
pro = list(map(int,pro))
np.sum(pro)

315140

## 测试集

In [87]:
df_test = pd.merge(df_test_queries,df_test_plans,how='left',on=['sid'])
df_test = pd.merge(df_test,df_profiles,how='left',on=['pid'])
df_test = df_test.drop(['pid','req_time','o','d','plan_time'],axis=1)
df_test = df_test.fillna(method='ffill')
df_test = df_test.fillna(method='bfill')


sid      94358
plans    94358
p0       94358
p1       94358
p2       94358
p3       94358
p4       94358
p5       94358
p6       94358
p7       94358
p8       94358
p9       94358
p10      94358
p11      94358
p12      94358
p13      94358
p14      94358
p15      94358
p16      94358
p17      94358
p18      94358
p19      94358
p20      94358
p21      94358
p22      94358
p23      94358
p24      94358
p25      94358
p26      94358
p27      94358
         ...  
p36      94358
p37      94358
p38      94358
p39      94358
p40      94358
p41      94358
p42      94358
p43      94358
p44      94358
p45      94358
p46      94358
p47      94358
p48      94358
p49      94358
p50      94358
p51      94358
p52      94358
p53      94358
p54      94358
p55      94358
p56      94358
p57      94358
p58      94358
p59      94358
p60      94358
p61      94358
p62      94358
p63      94358
p64      94358
p65      94358
Length: 68, dtype: int64

In [92]:
plan_ls_test = list(df_test.plans)
plan_ls_test = [json.loads(p) for p in plan_ls_test]
# 填补price缺失值，对没有缺失的数据用 gbdt拟合 ，预测缺失的price
x_test = []
y_test = []

for p in plan_ls_test:
    for pdc in p:
        if pdc['price']=="":
            dis = int(pdc['distance'])
            eta = int(pdc['eta'])
            mode = int(pdc['transport_mode'])
            pdc['price'] = gbdt.predict([[dis,eta,mode]])[0]
            

In [93]:
inf = 10000000
mode_dic_test = {}
for i in range(0,33):
    mode_dic_test[i]=[]
# for i in range(0,33):
#     mode_dic[i].append(inf)
# mode_dic
for p in plan_ls_test:
    for i in range(0,33):
        mode_dic_test[i].append(inf)
    for pdc in p:
            dis = int(pdc['distance'])
            eta = int(pdc['eta'])
            mode = int(pdc['transport_mode'])
            price = int(pdc['price'])
            mode_dic_test[(mode-1)*3+0][-1] = dis
            mode_dic_test[(mode-1)*3+1][-1] = eta
            mode_dic_test[(mode-1)*3+2][-1] = price

In [95]:
for i in range(0,33):
    df_test['plan'+str(i)] = mode_dic_test[i]
    mode_dic_test[i] = []
df_test['sid'] = df_test.sid.apply(int)
cols = list(df_test.columns)
for c in cols[2:68]:
    df_test[c] = df_test[c].apply(int)

In [97]:
df_test.head()

,sid,plans,p0,p1,p2,p3,p4,p5,p6,p7,...,plan23,plan24,plan25,plan26,plan27,plan28,plan29,plan30,plan31,plan32
0,1126541,"[{""distance"": 13610, ""price"": 400, ""eta"": 2388...",0,0,0,1,0,0,0,0,...,10000000,9175,2387,400,10000000,10000000,10000000,10000000,10000000,10000000
1,453685,"[{""distance"": 2858, ""price"": 200, ""eta"": 1798,...",1,0,0,0,0,0,0,1,...,10000000,10000000,10000000,10000000,10000000,10000000,10000000,10000000,10000000,10000000
2,2066073,"[{""distance"": 33453, ""price"": 800, ""eta"": 5624...",1,0,0,0,0,0,0,0,...,10000000,10000000,10000000,10000000,33101,4507,2800,10000000,10000000,10000000
3,452746,"[{""distance"": 4007, ""price"": 1500, ""eta"": 989,...",0,0,1,0,0,0,0,0,...,10000000,10000000,10000000,10000000,10000000,10000000,10000000,10000000,10000000,10000000
4,1431672,"[{""distance"": 692, ""price"": """", ""eta"": 591, ""t...",0,0,1,0,0,0,0,0,...,10000000,10000000,10000000,10000000,10000000,10000000,10000000,10000000,10000000,10000000


In [99]:
df_test['plans'] = df_test.plans.apply(plan2mode)
df_test.head()

,sid,plans,p0,p1,p2,p3,p4,p5,p6,p7,...,plan23,plan24,plan25,plan26,plan27,plan28,plan29,plan30,plan31,plan32
0,1126541,"[2, 3, 4, 9, 6, 7]",0,0,0,1,0,0,0,0,...,10000000,9175,2387,400,10000000,10000000,10000000,10000000,10000000,10000000
1,453685,"[1, 6, 3, 4, 1, 5]",1,0,0,0,0,0,0,1,...,10000000,10000000,10000000,10000000,10000000,10000000,10000000,10000000,10000000,10000000
2,2066073,"[7, 3, 4, 10, 1]",1,0,0,0,0,0,0,0,...,10000000,10000000,10000000,10000000,33101,4507,2800,10000000,10000000,10000000
3,452746,"[4, 3, 6, 2, 1, 5]",0,0,1,0,0,0,0,0,...,10000000,10000000,10000000,10000000,10000000,10000000,10000000,10000000,10000000,10000000
4,1431672,"[5, 3, 6, 1]",0,0,1,0,0,0,0,0,...,10000000,10000000,10000000,10000000,10000000,10000000,10000000,10000000,10000000,10000000


In [100]:
pred = bst.predict(xgb.DMatrix(df_test.drop(['sid','plans'],axis=1).values))
pred = pred.tolist()
pred_label = []
# sorted(list(zip(range(0,12),pred[0])),key = lambda x:x[1],reverse=True)
for i in range(len(pred)):
    sort_pred = sorted(list(zip(range(0,12),pred[i])),key = lambda x:x[1],reverse=True)
    for prob in sort_pred:
        if prob[0] in df_train['plans'][i] or prob[0] == 0:
            pred_label.append(prob[0])
            break


In [104]:
pred_label

[2,
 1,
 7,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 1,
 9,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 2,
 7,
 9,
 0,
 0,
 0,
 2,
 1,
 0,
 9,
 2,
 1,
 2,
 2,
 0,
 0,
 7,
 2,
 9,
 0,
 1,
 9,
 2,
 0,
 9,
 1,
 2,
 1,
 7,
 9,
 2,
 1,
 0,
 2,
 0,
 2,
 0,
 2,
 9,
 0,
 0,
 0,
 1,
 5,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 2,
 0,
 0,
 0,
 0,
 0,
 2,
 2,
 0,
 0,
 1,
 0,
 0,
 7,
 0,
 7,
 0,
 0,
 9,
 0,
 2,
 2,
 1,
 2,
 3,
 0,
 6,
 2,
 2,
 2,
 1,
 2,
 1,
 1,
 7,
 0,
 0,
 2,
 9,
 1,
 0,
 2,
 1,
 2,
 1,
 1,
 2,
 7,
 2,
 7,
 0,
 1,
 1,
 1,
 0,
 7,
 7,
 1,
 2,
 0,
 2,
 2,
 1,
 5,
 5,
 1,
 1,
 0,
 2,
 0,
 0,
 2,
 0,
 1,
 2,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 6,
 1,
 7,
 0,
 2,
 7,
 2,
 0,
 9,
 7,
 0,
 1,
 1,
 3,
 0,
 1,
 1,
 0,
 0,
 10,
 0,
 0,
 1,
 1,
 0,
 2,
 7,
 0,
 0,
 1,
 2,
 0,
 7,
 1,
 1,
 1,
 2,
 5,
 0,
 0,
 2,
 5,
 1,
 7,
 0,
 0,
 0,
 0,
 0,
 7,
 7,
 0,
 0,
 0,
 5,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 7,
 0,
 1,
 2,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 7,
 9,
 7,
 7,
 5,
 3,
 1,
 1,
 0,
 1,
 0,
 0,
 0,
 2,
 0,
 2,
 0,
 7,
 5,
 2,
 9,

In [102]:
df_test_res = pd.DataFrame({'sid':df_test.sid,'recommend_mode':pred_label})

In [103]:
df_test_res.to_csv(os.path.join(data_path,'ans_xgboost_1.csv'),quoting=1,index=False)